In [15]:
import utilities as utils

In [16]:
data_path_1: str = '../../../../Data/phase2'   
data_path_2: str = '../../../../Data/phase1/'  
 

data_set_1: list = [ 'copulagan_application_dataset_50k.csv', 
                     'copulagan_traffic_dataset_100k.csv',
                    ]

data_set_2: list = [ 'Traffic_type_seed.csv', 'Application_type_seed.csv' ] 


file_path_1       = utils.get_file_path(data_path_1)
file_path_2       = utils.get_file_path(data_path_2)
file_set_1 : list = list(map(file_path_1, data_set_1))
file_set_2 : list = list(map(file_path_2, data_set_2))

file_set : list   = file_set_1 + file_set_2 
data_set   : list = data_set_1 + data_set_2 
current_job: int  = 0

utils.data_set = data_set
utils.file_set = file_set

In [17]:
print(f'We will be using {len(file_set)} files:')
utils.pretty(file_set)

We will be using 4 files:
[   '../../../../Data/phase2/copulagan_application_dataset_50k.csv',
    '../../../../Data/phase2/copulagan_traffic_dataset_100k.csv',
    '../../../../Data/phase1/Traffic_type_seed.csv',
    '../../../../Data/phase1/Application_type_seed.csv']


In [18]:
copulagan_application_dataset_labels_50 = utils.examine_dataset(1)
copulagan_traffic_dataset_100 = utils.examine_dataset(2)
baseline_traffic_seed = utils.examine_dataset(3)
baseline_application_seed = utils.examine_dataset(4)

copulagan_traffic_dataset_100['Dataset'] = copulagan_traffic_dataset_100['Dataset'].drop(['Unnamed: 0'], axis = 1)
copulagan_application_dataset_labels_50['Dataset'] = copulagan_application_dataset_labels_50['Dataset'].drop(['Unnamed: 0'], axis = 1)

Dataset 1/4: We now look at ../../../../Data/phase2/copulagan_application_dataset_50k.csv


Loading Dataset: ../../../../Data/phase2/copulagan_application_dataset_50k.csv
	To Dataset Cache: ./cache/copulagan_application_dataset_50k.csv.pickle


        File:				../../../../Data/phase2/copulagan_application_dataset_50k.csv  
        Job Number:			1
        Shape:				(400000, 65)
        Samples:			400000 
        Features:			65
    
Dataset 2/4: We now look at ../../../../Data/phase2/copulagan_traffic_dataset_100k.csv


Loading Dataset: ../../../../Data/phase2/copulagan_traffic_dataset_100k.csv
	To Dataset Cache: ./cache/copulagan_traffic_dataset_100k.csv.pickle


        File:				../../../../Data/phase2/copulagan_traffic_dataset_100k.csv  
        Job Number:			2
        Shape:				(300000, 65)
        Samples:			300000 
        Features:			65
    
Dataset 3/4: We now look at ../../../../Data/phase1/Traffic_type_seed.csv


Loading Dataset: ../../../../Data/phase1/Traffic_type_seed.csv
	

In [19]:
def downsample(df: utils.pd.DataFrame, column_name: str, expected_sizes: dict) -> utils.pd.DataFrame():
    '''
        Function returns a new dataframe with the given column name and value
    '''
    new_dict = utils.pd.DataFrame()
    for item in df[column_name].unique():
        matching_values = df.loc[df[column_name] == item]
        if df[column_name].value_counts()[item] > expected_sizes[item]:
            new_dict = utils.pd.concat([new_dict, matching_values.sample(n = expected_sizes[item])])
        else:
            new_dict = utils.pd.concat([new_dict, matching_values])
    return new_dict

def random_sample(df: utils.pd.DataFrame, column_name: str, element_name: str, size : int) -> utils.pd.DataFrame():
    '''
        Function returns a new dataframe with the given column name and value
    '''
    new_df = df.loc[df[column_name] == element_name]
    return new_df.sample(n = size)

# Traffic Type Datasets Creation

In [20]:
expected_sizes = {"Regular" : 30000, "VPN" : 20000, "Tor": 10000}
copulagan_balanced_traffic_labels_dataset_30_20_10 = downsample(baseline_traffic_seed['Dataset'], 'Traffic Type', expected_sizes)
copulagan_balanced_traffic_labels_dataset_30_20_10 = utils.pd.concat([copulagan_balanced_traffic_labels_dataset_30_20_10, random_sample(copulagan_traffic_dataset_100['Dataset'], 'Traffic Type', 'Tor', 10000 - baseline_traffic_seed['Dataset']['Traffic Type'].value_counts()['Tor'])])

In [21]:
expected_sizes = {"Regular" : 92659, "VPN" : 92659, "Tor": 92659}
copulagan_balanced_traffic_labels_dataset_equal = downsample(baseline_traffic_seed['Dataset'], 'Traffic Type', expected_sizes)
copulagan_balanced_traffic_labels_dataset_equal = utils.pd.concat([copulagan_balanced_traffic_labels_dataset_equal, random_sample(copulagan_traffic_dataset_100['Dataset'], 'Traffic Type', 'VPN', 92659 - baseline_traffic_seed['Dataset']['Traffic Type'].value_counts()['VPN'])])
copulagan_balanced_traffic_labels_dataset_equal = utils.pd.concat([copulagan_balanced_traffic_labels_dataset_equal, random_sample(copulagan_traffic_dataset_100['Dataset'], 'Traffic Type', 'Tor', 92659 - baseline_traffic_seed['Dataset']['Traffic Type'].value_counts()['Tor'])])

# Application Types Data Creation

In [23]:
expected_sizes = {"p2p" : 30000, "browsing" : 30000, "audio-streaming": 30000, 'file-transfer' : 30000, 'chat': 30000, 'video-streaming': 30000, 'voip': 30000, 'email': 30000}
copulagan_balanced_application_dataset_labels_30_30_30 = downsample(baseline_application_seed['Dataset'], 'Application Type', expected_sizes)
copulagan_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([copulagan_balanced_application_dataset_labels_30_30_30, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'audio-streaming', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['audio-streaming'])])
copulagan_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([copulagan_balanced_application_dataset_labels_30_30_30, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'file-transfer', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['file-transfer'])])
copulagan_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([copulagan_balanced_application_dataset_labels_30_30_30, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'chat', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['chat'])])
copulagan_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([copulagan_balanced_application_dataset_labels_30_30_30, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'video-streaming', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['video-streaming'])])
copulagan_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([copulagan_balanced_application_dataset_labels_30_30_30, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'voip', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['voip'])])
copulagan_balanced_application_dataset_labels_30_30_30 = utils.pd.concat([copulagan_balanced_application_dataset_labels_30_30_30, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'email', 30000 - baseline_application_seed['Dataset']['Application Type'].value_counts()['email'])])


In [24]:
expected_sizes = {"p2p" : 48020, "browsing" : 48020, "audio-streaming": 48020, 'file-transfer' : 48020, 'chat': 48020, 'video-streaming': 48020, 'voip': 48020, 'email': 48020}
copulagan_balanced_application_dataset_labels_equal = downsample(baseline_application_seed['Dataset'], 'Application Type', expected_sizes)
copulagan_balanced_application_dataset_labels_equal = utils.pd.concat([copulagan_balanced_application_dataset_labels_equal, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'audio-streaming', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['audio-streaming'])])
copulagan_balanced_application_dataset_labels_equal = utils.pd.concat([copulagan_balanced_application_dataset_labels_equal, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'file-transfer', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['file-transfer'])])
copulagan_balanced_application_dataset_labels_equal = utils.pd.concat([copulagan_balanced_application_dataset_labels_equal, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'chat', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['chat'])])
copulagan_balanced_application_dataset_labels_equal = utils.pd.concat([copulagan_balanced_application_dataset_labels_equal, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'video-streaming', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['video-streaming'])])
copulagan_balanced_application_dataset_labels_equal = utils.pd.concat([copulagan_balanced_application_dataset_labels_equal, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'voip', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['voip'])])
copulagan_balanced_application_dataset_labels_equal = utils.pd.concat([copulagan_balanced_application_dataset_labels_equal, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'email', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['email'])])
copulagan_balanced_application_dataset_labels_equal = utils.pd.concat([copulagan_balanced_application_dataset_labels_equal, random_sample(copulagan_application_dataset_labels_50['Dataset'], 'Application Type', 'browsing', 48020 - baseline_application_seed['Dataset']['Application Type'].value_counts()['browsing'])])



In [25]:
copulagan_balanced_traffic_labels_dataset_30_20_10.to_csv('./synthetic/copulagan_traffic_30_20_10.csv', index=False)
copulagan_balanced_traffic_labels_dataset_equal.to_csv('./synthetic/copulagan_traffic_upsample_to_majority.csv', index=False)
copulagan_balanced_application_dataset_labels_30_30_30.to_csv('./synthetic/copulagan_application_30000.csv', index=False)
copulagan_balanced_application_dataset_labels_equal.to_csv('./synthetic/copulagan_application_upsample_to_majority.csv', index=False)

In [26]:
print(f'Last Execution: {utils.datetime.datetime.now()}')
assert False, 'Nothing after this point is included in the study'

Last Execution: 2022-05-01 09:13:53.267341


AssertionError: Nothing after this point is included in the study